In [1]:
!pip install ratsnlp -q
# -q 는 quiet 모드..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.6 MB/s eta 0:00:00


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언

인자(argument)의 역할과 내용
- pretrained_model_name : 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_model_dir : 파인튜닝한 모델의 체크포인트 저장 위치.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무 것도 입력하지 않으면 128입니다.

In [3]:
# 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls1",
    # /gdrive/MyDrive/nlpbook/checkpoint-doccls1 -> 경로 확인 완료
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls1/epoch=0-val_loss=0.28.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

- fine_tuned_model_ckpt['state_dict']: 파인튜닝된 모델의 상태를 가지고 있는 딕셔너리. 모델의 각 레이어에 대한 가중치와 bias 값들 저장
- 'model.classifier.bias': 분류기의 bias 텐서를 가져온다.이 분류기는 각 클래스에 대한 확률을 출력하며, bias 텐서 크기는 분류 클래스 수와 동일(즉, bias 텐서 크기는 분류 클래스 수에 따라 정해진다는 의미)
- .shape.numel(): 텐서의 모든 원소의 개수를 반환. 이 경우에는 bias 텐서의 원소의 개수, 즉 클래스의 수를 반환


/usr/local/lib/python3.10/dist-packages/transformers/models/bert/configuration_bert.py
  - 72번째줄에..<br>
  class BertConfig(PretrainedConfig):

/usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py
  - 1517번째줄에 ..<br>
  class BertForSequenceClassification(BertPreTrainedModel):

In [5]:
import torch
from transformers import BertConfig, BertForSequenceClassification
# 체크포인트 로드
# downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls1/epoch=0-val_loss=0.27-v1.ckpt
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu") # 체크포인트를 CPU 메모리에 로드
)
# 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델의 설정 값들을 읽어 들일 수 있음
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    # 파인튜닝된 BERT 모델의 분류기가 가지고 있는 바이어스 파라미터의 크기를 기반으로, 모델이 처리해야 할 클래스(레이블)의 총 개수를 동적으로 설정
    # 바이어스 파라미터의 총 개수는 모델이 예측해야 할 클래스의 총 개수와 직접적으로 일치합니다.
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
    # numel() 는 num-elements..(number of elements) 개수 구하는 함수.
)
# 초기화한 BERT 모델에 체크포인트(fine_tuned_model_ckpt)를 읽어들이게 된다
model = BertForSequenceClassification(pretrained_model_config) # birt 모델 초기화
# 파인튜닝된 모델의 체크포인트 상태 딕셔너리를 현재 모델의 상태 딕셔너리 키 형식에 맞게 조정하여,
# 현재 모델에 파인튜닝된 가중치를 정확히 매핑하고 로드하는 데 사용
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()}) # 체크포인트 주입
# load_state_dict 함수는 모델의 파라미터(가중치)를 로드하는 데 사용
# items을 빼면 키와 밸류만 남음.
# "model." 접두사를 제거하여, 현재 모델의 파라미터 이름과 일치시키는 과정. 파인 튜닝된 모델과 현재 모델의 파라미터 이름이 조금 다를 수 있기 때문에 필요한 단계

# 모델이 평가 모드로 전환. 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 한다.
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
# 토크나이저 초기화
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 인퍼런스 함수 선언
- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만든다.
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다. - 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태인데요. 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꾼다.
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만든다.

In [7]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True, # 남은것 잘라버리기
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        # dim=1은 각 입력 샘플에 대해 각 클래스의 로짓을 확률로 변환하라는 의미. prob = [[0.2, 0.8]]
        prob = outputs.logits.softmax(dim=1)
        # 긍정 및 부정 클래스에 대한 확률을 계산하고, 이를 반올림하여 4자리 소수점까지 표시
        positive_prob = round(prob[0][1].item(), 4) # 인덱스1 -> positive
        negative_prob = round(prob[0][0].item(), 4) # 인덱스0 -> negative
        # 확률이 더 높은 클래스의 인데스 기반으로 최종 예측 결과(긍정 또는 부정)를 결정
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{round(negative_prob * 100,2)}%",
    }

In [46]:
sentence = '어린이 영화인줄 알고 봤는데 너무 잔인해서 충격적이었어요.'
inference_fn(sentence)

{'sentence': '어린이 영화인줄 알고 봤는데 너무 잔인해서 충격적이었어요.',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.9001',
 'negative_data': '부정 0.0999',
 'positive_width': '90.01%',
 'negative_width': '9.99%'}

In [9]:
sentence = '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'
inference_fn(sentence)

{'sentence': '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.0536',
 'negative_data': '부정 0.9464',
 'positive_width': '5.36%',
 'negative_width': '94.64%'}

In [ ]:
!cp -r /gdrive/MyDrive/nlpbook/checkpoint-doccls1 /gdrive/MyDrive/kdt_231026/m7_nlp응용/data/